### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [16]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function parabola(x)
    a = 0.13625841750841752
    b = 0.189709595959596
    c = 0.22
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [17]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [27]:
startI = -1.705
endI = -1.75
lengthI = 200

rangeI = range( startI, endI, length = lengthI )

step_to_left = rangeI.step.hi

-0.00022613065326632098

In [28]:
file = "2d_map_parabola_zoom_reverse.txt"

"2d_map_parabola_zoom_reverse.txt"

In [ ]:
for (index_I0, I0) in enumerate(rangeI)
    
        if index_I0 == 1
            global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
        end
        
        U0 = parabola(I0)
    
        print_output(index_I0, I0, U0, initial_condition_function)
    
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    
        ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_function = tr[end, :]
    
        println("last point: $initial_condition_function");flush(stdout)
    
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 4, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
    
        println("----------------------")
    
        if I0 != -1.75
            move_to_left( I0, U0, initial_condition_function, step_to_left )
        end
        
end

index: 1
I0: -1.705
U0: 0.2926517650462963
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [3.07618504041362, 0.48970947753123895, 0.7789842956814212, 0.06464758424385432]
LSE: [-0.004207965412879482, -0.5555557904803415, -0.8523111511813979, -7.832413345689426]
----------------------

<<<<<<<<<<<<<<<<<<<<<
MOVE TO LEFT
 <- 
index: 1
I0: -1.7052261306532663
U0: 0.2926517650462963
initial condition: [3.07618504041362, 0.48970947753123895, 0.7789842956814212, 0.06464758424385432]
last point: [1.6873002023251409, 0.840526783468722, 0.67799179899348, 0.0657879427671468]
LSE: [0.008138906496881907, -0.5555555175634755, -0.8392423689628302, -7.840274103915949]
_________________

 <- 
index: 2
I0: -1.7054522613065326
U0: 0.2926517650462963
initial condition: [1.6873002023251409, 0.840526783468722, 0.67799179899348, 0.0657879427671468]
last point: [1.299878915807506, 0.8225566751658245, 0.6953539902419008, 0.06222909069044837]
LSE: [0.006524111227003303, -0.5555554871013596, -0.83407961305

LSE: [-0.00199194387726702, -0.5555557990424712, -0.6235996585595621, -7.8215095726073995]
_________________

 <- 
index: 23
I0: -1.7102010050251253
U0: 0.2926517650462963
initial condition: [24.307717687188763, 0.4607311125450563, 0.7363618127469667, 0.0765155814266586]
last point: [1.8007412843198873, 0.8458125601422517, 0.6689655509812497, 0.07407678312906495]
LSE: [0.005165159747376775, -0.5555558631019787, -0.6102662438660024, -7.818003530842579]
_________________

 <- 
index: 24
I0: -1.7104271356783918
U0: 0.2926517650462963
initial condition: [1.8007412843198873, 0.8458125601422517, 0.6689655509812497, 0.07407678312906495]
last point: [1.0615782190588634, 0.7850788766733944, 0.7120358752539971, 0.06723354337431096]
LSE: [0.0013127771790852004, -0.5555560038104016, -0.5954039518635055, -7.816273384819486]
_________________

 <- 
index: 25
I0: -1.710653266331658
U0: 0.2926517650462963
initial condition: [1.0615782190588634, 0.7850788766733944, 0.7120358752539971, 0.067233543374310

U0: 0.2926517650462963
initial condition: [1.3291566585854477, 0.6071463995829846, 0.7521676087639078, 0.084261287053484]
last point: [1.259050683975651, 0.8319277600031207, 0.6851261978777319, 0.08370724661279205]
LSE: [0.007023588529507679, -0.01023169756659456, -2.0553509437081763, -11.640024250576968]
_________________

 <- 
index: 46
I0: -1.7154020100502507
U0: 0.2926517650462963
initial condition: [1.259050683975651, 0.8319277600031207, 0.6851261978777319, 0.08370724661279205]
last point: [1.4450222836286388, 0.8944426570448517, 0.6233573966208794, 0.4982792430282382]
LSE: [-0.0026059545324560104, -0.03554608727607715, -2.3055422934908885, -12.341257382358386]
_________________

 <- 
index: 47
I0: -1.715628140703517
U0: 0.2926517650462963
initial condition: [1.4450222836286388, 0.8944426570448517, 0.6233573966208794, 0.4982792430282382]
last point: [1.9617829331977297, 0.8457851841365635, 0.6657809586420061, 0.3543110038034364]
LSE: [0.00981444304082429, -0.007454073517852348, -2

LSE: [-0.31243149409998966, -0.3058838443354213, -11.368086161980077, -15.56994464777529]
_________________

 <- 
index: 68
I0: -1.72037688442211
U0: 0.2926517650462963
initial condition: [0.9531876671703925, 0.9246684709501136, 0.5697981824304634, 0.48895863936599854]
last point: [0.953030485185392, 0.9246737527205928, 0.5698489461963128, 0.488978220148458]
LSE: [-0.309328192677197, -0.3091750712872849, -11.366600902178831, -15.572562837888773]
_________________

 <- 
index: 69
I0: -1.7206030150753762
U0: 0.2926517650462963
initial condition: [0.953030485185392, 0.9246737527205928, 0.5698489461963128, 0.488978220148458]
last point: [0.9528733257097852, 0.9246790340799098, 0.5698997171054181, 0.48899779890648654]


In [36]:
function move_to_left( I0, U0, initial_condition_function, step_to_left )
    
    println("");flush(stdout)
    println("<<<<<<<<<<<<<<<<<<<<<");flush(stdout)
    
    println("MOVE TO LEFT")
        
    range_to_left_border = range( I0 + step_to_left, endI, step = step_to_left )
        
    for (index_left, I0_left) in enumerate( range_to_left_border )
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
        end
            
        println(" <- ")
        print_output(index_left, I0_left, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0_left $U0 $(λλ[1])")
        end
        separate()
    end
end

move_to_left (generic function with 1 method)